# 1.0 - IMPORTS

In [48]:
# Adicione o caminho do diretório raiz do projeto
import os, sys; sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import importlib
import plotly.express as px

from packages import etl
from packages import utils
from packages import layouts

importlib.reload(etl.extract)
importlib.reload(etl.load)
importlib.reload(etl.transform)
importlib.reload(utils)
importlib.reload(layouts)

pd.set_option('display.max_columns', None);

# 2.0 - ETL

In [34]:
# Extrair os dados
df_raw = etl.extract.csv('../data/raw/zomato.csv')

# Transformar os dados
df = etl.transform.dataframe(df_raw)

# Salvar os dados transformados (opcional)
etl.load.save(df, '../data/processed/zomato.csv')

# Visualizar os dados
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,rating_color_name,rating_name,country_name,price_type
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,excellent,Philippines,expensive
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,Botswana Pula(P),0,0,0,4,4.7,3F7E00,Excellent,469,darkgreen,excellent,Philippines,gourmet
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,800,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,867,green,great,Philippines,expensive
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,858,green,great,Philippines,expensive
5,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,700,Botswana Pula(P),0,0,0,3,4.7,3F7E00,Excellent,930,darkgreen,excellent,Philippines,expensive


In [71]:
df[df.aggregate_rating == 0][[
    'country_name',
    'city',
    'cuisines',
    'aggregate_rating'
]].groupby([
    'country_name',
    'city',
    'cuisines'
]).count()

,country_name,city,cuisines,aggregate_rating
105,Brazil,Brasília,Brazilian,0.0
106,Brazil,Brasília,Bar Food,0.0
107,Brazil,Brasília,Seafood,0.0
123,Brazil,Brasília,Japanese,0.0
124,Brazil,Brasília,Tex-Mex,0.0
...,...,...,...,...
6798,England,Manchester,Indian,0.0
7009,South Africa,Clarens,Cafe,0.0
7010,South Africa,Clarens,Grill,0.0
7011,South Africa,Clarens,Drinks Only,0.0


# 3.0 - Desafio

## 3.1 - Geral

### 1. Quantos restaurantes únicos estão registrados?

In [ ]:
df.restaurant_id.nunique()

### 2. Quantos países únicos estão registrados?

In [ ]:
df.country_name.nunique()

### 3. Quantas cidades únicas estão registradas?

In [ ]:
df.city.nunique()

### 4. Qual o total de avaliações feitas?

In [ ]:
df.votes.sum()

### 5. Qual o total de tipos de culinária registrados?

In [ ]:
df.cuisines.nunique()

## 3.2 - País

### 1. Qual o nome do país que possui mais cidades registradas?

In [ ]:
(df[['country_name', 
    'city']].groupby([
    'country_name'])
    .count()
    .reset_index()
    .sort_values('city', ascending=False))

### 2. Qual o nome do país que possui mais restaurantes registrados?

In [ ]:
(df[['country_name', 
    'restaurant_id']].groupby([
    'country_name'])
    .count()
    .reset_index()
    .sort_values('restaurant_id', ascending=False))

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [ ]:
df_aux = df[df['price_range'] == 4]

(df_aux[['country_name', 'city']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['city'], ascending=False))

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
(df_aux[['country_name', 'cuisines']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['cuisines'], ascending=False))

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
(df_aux[['country_name', 'votes']]
 .groupby(['country_name'])
 .sum()
 .reset_index()
 .sort_values(['votes'], ascending=False))

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
df__delivery = df[df['is_delivering_now'] == 1]
(df__delivery[['country_name', 'is_delivering_now']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['is_delivering_now'], ascending=False)
 .head(1))

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [ ]:
df_aux = df[df['has_table_booking'] == 1]

(df_aux[['country_name', 'has_table_booking']]
 .groupby(['country_name'])
 .count()
 .reset_index()
 .sort_values(['has_table_booking'], ascending=False))

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [ ]:
df_avg_votes = (df_aux[['country_name', 'votes']]
          .groupby(['country_name'])
          .mean()
          .reset_index()
          .sort_values(['votes'], ascending=False))
df_avg_votes.columns = ['country_name', 'avg_votes']
df_avg_votes

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [ ]:
(df[['country_name', 'aggregate_rating']]
 .groupby(['country_name'])
 .mean()
 .sort_values('aggregate_rating', ascending=False)
 .reset_index()
 .head(1))

### 10. Qual a média de preço de um prato para dois por país?

In [ ]:
df.average_cost_for_two.mean()

## 3.3 - Cidade

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
(df[['restaurant_id', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['restaurant_id'], ascending=False))

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
df_votes = df[df['votes'] == 4]
(df_votes[['votes', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['votes'], ascending=False)
 .head())

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [ ]:
df_votes = df[df['aggregate_rating'] < 2.5]
(df_votes[['aggregate_rating', 'city']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['aggregate_rating'], ascending=False)
 .head())

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
(df[['restaurant_id', 'city', 'average_cost_for_two']]
 .groupby(['city', 'average_cost_for_two'])
 .count()
 .sort_values(['average_cost_for_two'], ascending=False)
 .head(1))

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
(df[['city', 'cuisines']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['cuisines'], ascending=False)
 .head(1))

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
df_booking = df[df['has_table_booking'] == 1]
(df_booking[['city', 'has_table_booking']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['has_table_booking'], ascending=False)
 .head(1))

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
df_booking = df[df['is_delivering_now'] == 1]
(df_booking[['city', 'is_delivering_now']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['is_delivering_now'], ascending=False)
 .head(1))

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
df_booking = df[df['has_online_delivery'] == 1]
(df_booking[['city', 'has_online_delivery']]
 .groupby(['city'])
 .count()
 .reset_index()
 .sort_values(['has_online_delivery'], ascending=False)
 .head(1))

## 3.4 - Restaurante

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [ ]:
(df[['restaurant_name', 'votes']]
 .groupby(['restaurant_name'])
 .sum()
 .reset_index()
 .sort_values('votes', ascending=False)
 .head(1))

### 2. Qual o nome do restaurante com a maior nota média?

In [ ]:
(df[['restaurant_name', 'aggregate_rating']]
 .sort_values('aggregate_rating', ascending=False)
 .head(1))

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [ ]:
(df[['restaurant_name', 'average_cost_for_two']]
 .sort_values(['average_cost_for_two'], ascending=False)
 .head(1))

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [ ]:
df_brazilian = df.loc[df['cuisines'] == 'Brazilian', :].copy()
df_brazilian['index'] = df_brazilian.index
df_brazilian = df_brazilian.reset_index()
(df_brazilian[['index', 'restaurant_name', 'aggregate_rating']]
 .sort_values(['aggregate_rating', 'index'], ascending=[False, True])
 .head(1))

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [ ]:
df_brazil = (df.loc[
    (df['cuisines'] == 'Brazilian') & 
    (df['country_name'] == 'Brazil'), :]
    .copy())

df_brazil['index'] = df_brazil.index
df_brazil = df_brazil.reset_index()
(df_brazil[['index', 'country_name', 'restaurant_name', 'aggregate_rating']]
 .sort_values(['aggregate_rating', 'index'], ascending=[False, True])
 .head(1))

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [ ]:
df_delivery = df.loc[df['has_online_delivery'] == 1]
(df_delivery[['restaurant_name', 'has_online_delivery','votes']]
 .groupby(['restaurant_name', 'has_online_delivery'], observed=False)
 .mean()
 .sort_values(['votes'], ascending=False)
 .reset_index()
 .head(1))


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [ ]:
(df[['restaurant_name', 'has_table_booking', 'average_cost_for_two']]
 .groupby(['has_table_booking', 'restaurant_name'], observed=False)
 .mean()
 .sort_values(['average_cost_for_two'], ascending=False)
 .reset_index()
 .head(2))


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [ ]:
df_usa_aux = (df[
    (df['country_name'] == 'United States of America') &
    ((df['cuisines'] == 'Japanese') |
    (df['restaurant_name'] == 'Blue Tractor BBQ & Brewery'))
])
(df_usa_aux[['country_name', 
         'cuisines',
         'restaurant_name',
         'average_cost_for_two'
         ]]
        .groupby(['country_name', 
                   'cuisines', 
                   'restaurant_name'
                   ])
        .mean()
        .sort_values([
            'cuisines', 
            'average_cost_for_two'
        ], ascending=[True, False])
        .head(2))

## 3.5 - Tipos de Culinária

### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [68]:
def restaurant_avg_rating(data, cuisine, ascending, total_lines=1):
    df_filtered = data[data['cuisines'] == cuisine].copy()
    df_filtered['index'] = df_filtered.index
    df_result = (df_filtered[['index', 'cuisines', 'restaurant_name', 'aggregate_rating', 'country_name']]
                    .sort_values(['aggregate_rating', 'index'], ascending=[ascending, True])
                    .head(total_lines))
    df_result = df_result.drop('index', axis=1)
    if not df_result.empty:
        return (
            df_result.cuisines.iloc[0],
            df_result.country_name.iloc[0],
            df_result.restaurant_name.iloc[0],
            df_result.aggregate_rating.iloc[0]
        )
    else:
        return None, None, None, None

df_countries = df.loc[df.country_name.isin(['England']), :]
# cuisine, country, restaurant, rating = restaurant_avg_rating(data=df_countries, cuisine='Italian', ascending=False)
# cuisine, country, restaurant, rating = restaurant_avg_rating(data=df_countries, cuisine='American', ascending=False)
cuisine, country, restaurant, rating = restaurant_avg_rating(data=df_countries, cuisine='Arabian', ascending=False)
# cuisine, country, restaurant, rating = restaurant_avg_rating(data=df_countries, cuisine='Japanese', ascending=False)
# cuisine, country, restaurant, rating = restaurant_avg_rating(data=df_countries, cuisine='Brazilian', ascending=False)

In [59]:
utils.restaurant_avg_rating(data=df, cuisine='Italian', ascending=False)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,rating_color_name,rating_name,country_name,price_type
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,excellent,Philippines,expensive
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,Botswana Pula(P),0,0,0,4,4.7,3F7E00,Excellent,469,darkgreen,excellent,Philippines,gourmet
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,800,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,867,green,great,Philippines,expensive
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,700,Botswana Pula(P),0,0,0,3,4.4,5BA829,Very Good,858,green,great,Philippines,expensive
5,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,700,Botswana Pula(P),0,0,0,3,4.7,3F7E00,Excellent,930,darkgreen,excellent,Philippines,expensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,300,Turkish Lira(TL),0,0,0,4,4.3,5BA829,Very Good,1367,green,great,Turkey,gourmet
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,45,Turkish Lira(TL),0,0,0,2,4.5,3F7E00,Excellent,1172,darkgreen,excellent,Turkey,normal
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,140,Turkish Lira(TL),0,0,0,4,4.2,5BA829,Very Good,1160,green,great,Turkey,gourmet
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,95,Turkish Lira(TL),0,0,0,3,4.0,5BA829,Very Good,770,green,great,Turkey,expensive


### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Italian', ascending=True)

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='American', ascending=False)

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='American', ascending=True)

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Arabian', ascending=False)

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Arabian', ascending=True)

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Japanese', ascending=False)

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Japanese', ascending=True)

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Home-made', ascending=False)

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
Utils.restaurant_avg_rating(data=df, cuisine='Home-made', ascending=True)

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [ ]:
df_cuis_filtered = df[['cuisines', 'average_cost_for_two']].copy()
df_cuis_filtered['index'] = df_cuis_filtered.index
df_cuis = (df_cuis_filtered.sort_values(['average_cost_for_two', 'index'], ascending=[False, True])
        .head(1))
df_cuis = df_cuis.drop('index', axis=1)
df_cuis

### 12. Qual o tipo de culinária que possui a maior nota média?

In [ ]:
df_cuis_filtered = df[['cuisines', 'aggregate_rating']].copy()
df_cuis_filtered['index'] = df_cuis_filtered.index
df_cuis = (df_cuis_filtered.sort_values(['aggregate_rating', 'index'], ascending=[False, True])
        .head(1))
df_cuis = df_cuis.drop('index', axis=1)
df_cuis

### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [ ]:

df_deliv = df[(df['has_online_delivery'] == 1) & (df['is_delivering_now'] == 1)]

df_deliv_result = (df_deliv[['restaurant_id', 'cuisines']]
 .groupby('cuisines')
 .count()
 .sort_values('restaurant_id', ascending=False)
 .reset_index()
 .head(1))
df_deliv_result.columns = ['cuisines', 'total de restaurantes']
df_deliv_result